In [2]:
import numpy as np

In [5]:
def split_sequences(sequences,n_steps):
    X = list()
    Y = list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        seq_x = sequences[i:end_ix, :]
        seq_y = sequences[end_ix, : ]
        X.append(seq_x)
        Y.append(seq_y)
    return np.array(X),np.array(Y)

In [12]:
# define input sequence
seq1 = np.array([10,20,30,40,50,60,70,80,90])
seq2 = np.array([15,25,35,45,55,65,75,85,95])

In [13]:
out_seq = np.array([seq1[i]+seq2[i] for i in range(len(seq1))])

In [14]:
# convert to [rows, columns] structure
seq1 = seq1.reshape((len(seq1),1))
seq2 = seq2.reshape((len(seq2),1))
out_seq = out_seq.reshape((len(out_seq),1))

In [20]:
print(seq1.shape)

(9, 1)


In [15]:
# horizontally stack columns
dataset = np.hstack((seq1,seq2,out_seq))

In [25]:
dataset

array([[ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

In [16]:
# choose a number of time steps
n_steps = 3

In [18]:
X,Y = split_sequences(dataset,n_steps)

The shape of X is three-dimensional, including the number of samples (6), the number of time steps chosen per sample (3), and the number of parallel time series or features (3).

The shape of y is two-dimensional as we might expect for the number of samples (6) and the number of time variables per sample to be predicted (3).

In [21]:
X.shape

(6, 3, 3)

In [22]:
Y.shape

(6, 3)

In [19]:
n_features = X.shape[2]

In [23]:
X

array([[[ 10,  15,  25],
        [ 20,  25,  45],
        [ 30,  35,  65]],

       [[ 20,  25,  45],
        [ 30,  35,  65],
        [ 40,  45,  85]],

       [[ 30,  35,  65],
        [ 40,  45,  85],
        [ 50,  55, 105]],

       [[ 40,  45,  85],
        [ 50,  55, 105],
        [ 60,  65, 125]],

       [[ 50,  55, 105],
        [ 60,  65, 125],
        [ 70,  75, 145]],

       [[ 60,  65, 125],
        [ 70,  75, 145],
        [ 80,  85, 165]]])

In [24]:
Y

array([[ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

# Model Building

In [27]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# Multi - parallel :

An alternate time series problem is the case where there are multiple parallel time series and a value must be predicted for each.

In [52]:
model = Sequential()
model.add(LSTM(100,activation = 'relu',return_sequences = True,input_shape = (n_steps,n_features)))
model.add(LSTM(100,activation = 'relu'))
model.add(Dense(n_features))
model.compile(optimizer = 'adam',loss = 'mse')

In [30]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 3, 100)            41600     
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 122,303
Trainable params: 122,303
Non-trainable params: 0
_________________________________________________________________


In [44]:
model.fit(X,Y,epochs = 100)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
6/6 [==============================] - 2s 393ms/step - loss: 9830.7021
Epoch 2/100
6/6 [==============================] - 0s 2ms/step - loss: 9517.7646
Epoch 3/100
6/6 [==============================] - 0s 1ms/step - loss: 9279.7910
Epoch 4/100
6/6 [==============================] - 0s 2ms/step - loss: 9053.2295
Epoch 5/100
6/6 [==============================] - 0s 2ms/step - loss: 8857.0068
Epoch 6/100
6/6 [==============================] - 0s 1ms/step - loss: 8604.1807
Epoch 7/100
6/6 [==============================] - 0s 2ms/step - loss: 8288.1689
Epoch 8/100
6/6 [==============================] - 0s 1ms/step - loss: 7892.6094
Epoch 9/100
6/6 [==============================] - 0s 1ms/step - loss: 7456.0054
Epoch 10/100
6/6 [==============================] - 0s 1ms/step - loss: 7020.2993
Epoch 11/100
6/6 [==============================] - 0s 1ms/step - loss: 6574.1470
Epoch 12/10

Epoch 99/100
6/6 [==============================] - 0s 1000us/step - loss: 0.0730
Epoch 100/100
6/6 [==============================] - 0s 1ms/step - loss: 0.0677


# Prediction

In [45]:
x_input = np.array([[30,35,65],[40,45,85],[50,55,105]])

In [46]:
x_input = x_input.reshape((1,n_steps,n_features))

The shape of the input for making a single prediction must be 1 sample, 3 time steps, and 3 features, or [1, 3, 3]

In [47]:
x_input.shape

(1, 3, 3)

In [48]:
x_input

array([[[ 30,  35,  65],
        [ 40,  45,  85],
        [ 50,  55, 105]]])

In [49]:
pred = model.predict(x_input)

In [50]:
pred

array([[ 60.156055,  64.98491 , 124.96425 ]], dtype=float32)